In [8]:
# PRICE HISTORY SCRIPT
# This script is responsible for getting the price
# history of any symbol passed to the constructor
from src.urls import TDA_BASE
from config.secrets import TDA_APIKEY, PERIOD, PERIODTYPE, FREQUENCY, FREQUENCYTYPE
import requests
import multiprocessing
import pandas as pd
from models.mysql_db import create_pricehistory_engine, generate_symbols, _select_symbols
from loguru import logger
import os.path
import time

In [9]:
# CREATE THE LOGGER FOR THIS SCRIPT:
log_path = str(os.path.curdir) + '/logs/'
base_fmt = "[{time:YYYY-MM-DD at HH:mm:ss}]|[{name}-<lvl>{message}</lvl>]"
logger.add(log_path+"pricehistory.log", format=base_fmt, level="DEBUG", rotation="20 MB",
           colorize=True, enqueue=True, catch=True)

3

In [10]:
"""
=================================================================================
THE PRICE HISTORY CLASS|
-----------------------+
This will be responsible for getting the price history for every symbol
stored in the database. Using a generator function in the models.mysql_db
script, I will be able to generate one symbol at a time and then execute
the insert_price_data() method.

I am going to try two different approaches. First, I will try the map()
function to see if it can handle executing the function on that long of
a list.

If that doesn't work, I will try to utilize the generator function to
generate one symbol at a time and only execute the insert method one
stock at a time.
=================================================================================
"""

"\n=================================================================================\nTHE PRICE HISTORY CLASS|\n-----------------------+\nThis will be responsible for getting the price history for every symbol\nstored in the database. Using a generator function in the models.mysql_db\nscript, I will be able to generate one symbol at a time and then execute\nthe insert_price_data() method.\n\nI am going to try two different approaches. First, I will try the map()\nfunction to see if it can handle executing the function on that long of\na list.\n\nIf that doesn't work, I will try to utilize the generator function to\ngenerate one symbol at a time and only execute the insert method one\nstock at a time.\n=================================================================================\n"

In [11]:
class PriceHistory:

    def __init__(self, **params):
        self.params = params  # params are set at bottom, imported from config.ini file
        self.table_name = self._set_table_name()
        self.pricehistory_engine = create_pricehistory_engine()

    def _set_table_name(self):
        """
        Simple way to set the name of the table to save the price data to
        dynamically. This way no matter what the params are, the data will
        be saved to correct table.
        """
        if int(FREQUENCY) > 1:
            name = f"_{FREQUENCY}_{FREQUENCYTYPE}_data"
            logger.info("[+] Storing Pricehistory data to table: {}", name)
            return name
        else:
            name = f"one_{FREQUENCYTYPE}_data"
            logger.info("[+] Storing Pricehistory data to table: {}", name)
            return name

    def data(self, stock):
        """
        :param symbol: company symbol/ticker
        :Example: MSFT 10 day minute 10

        :returns:
        raw json data (Open, High, Low, close, Volume, and Time (epoch time))
        """
        url = TDA_BASE + f"marketdata/{stock}/pricehistory"

        params = {
            'period': self.params['params']['period'],
            'periodType': self.params['params']['periodType'],
            'frequency': self.params['params']['frequency'],
            'frequencyType': self.params['params']['frequencyType'],
        }

        # Other users will need their own TD Ameritrade API Key
        params.update({"apikey": TDA_APIKEY})

        # request price history data
        req = requests.get(url, params=params).json()

        candles = dict(req)  # turn candles into a dict() type
        extracted_candles_list = candles["candles"]
        symbol = candles["symbol"]  # symbol of the compan's price data

        # Create data frame from extracted data
        df = pd.DataFrame.from_dict(extracted_candles_list, orient="columns")
        df.rename(columns={"datetime": "unix"}, inplace=True)
        df["unix"] = [x for x in df["unix"] // 10 ** 3]

        # This is to insert the companies symbol into the data frame
        # in every row next to the unix_time so that I can identify
        # who the data belongs to.
        df["symbol"] = symbol

        return df

    def insert_price_data(self, stock):
        try:
            data = self.data(stock)
            table = self.table_name
            engine = self.pricehistory_engine
            data.to_sql(name=table, con=engine,
                        if_exists='append', index=False)
            logger.info("{} inserted successfully!", stock)
        except Exception as e:
            logger.error("Error Caused Due to {}", e)
            
    def execute_data(self, stock):
        
        pass

    def execute_main(self):
        symbols = _select_symbols()
        # symbol = generate_symbols()
        pool = multiprocessing.Pool(processes=4)
        try:
            count = 0
            while True:
                try:
                    # stock = next(symbol)
                    # self.insert_price_data(stock)
                    # logger.info("[{}]<green>Price History Data Inserted Successfully</green>", stock)
                    logger.info("[<green>Price History Map Function Started</green>]")
                    pool.map(self.insert_price_data, symbols)
                    count += 1
                except KeyError as ke:
                    logger.error("Failed to insert: Due to {}", ke)
                    continue
                except StopIteration as si:
                    logger.info("{} No More Stocks to Get Data for", si)
                    continue
        except ValueError as ve:
            logger.error("Error Caused Due to {}", ve)
            if ve:
                engine = self.pricehistory_engine
                stmt = "DROP TABLE IF EXISTS {table}".format(self.table_name)
                engine.execute(stmt)
                logger.info("SQL Statement {} Executed...", stmt)
        except Exception as e:
            logger.error("Error Caused Due to {}", e)
        finally:
            logger.info("[{}] Stocks Inserted Successfully!", count)

In [12]:
class ProcessPricehistory(multiprocessing.Process):
    
    def __init__(self, task):
        super(ProcessPricehistory, self).__init__()
        self.task = task

    def run(self):
        # THIS WILL EXECUTE THE MAIN METHOD IN QUOTE USING MAP AND THREADED POOL PROCESSES:
        pass

In [13]:
params = {
    'symbol': 'stock',
    'period': PERIOD,
    'periodType': PERIODTYPE,
    'frequency': FREQUENCY,
    'frequencyType': FREQUENCYTYPE,
}

In [14]:
price_history = PriceHistory(params=params)
logger.info(
    "PriceHistory Object Initialized: Table {} created", price_history.table_name)


2022-05-15 00:06:01.862 | INFO     | __main__:_set_table_name:20 - [+] Storing Pricehistory data to table: one_minute_data
2022-05-15 00:06:01.864 | INFO     | models.mysql_db:create_pricehistory_engine:33 - [+] pricehistory database engine created successfully
2022-05-15 00:06:01.866 | INFO     | __main__:<cell line: 2>:2 - PriceHistory Object Initialized: Table one_minute_data created


In [15]:
price_history.execute_main()

2022-05-15 00:06:04.009 | INFO     | __main__:execute_main:84 - [<green>Price History Map Function Started</green>]
2022-05-15 00:06:04.012 | ERROR    | __main__:execute_main:101 - Error Caused Due to Can't pickle local object 'create_engine.<locals>.connect'
2022-05-15 00:06:04.014 | INFO     | __main__:execute_main:103 - [0] Stocks Inserted Successfully!
